In [ ]:
!pip install pydicom

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pydicom
import cv2

from pydicom.pixel_data_handlers.util import apply_voi_lut
from skimage import data
from skimage.draw import ellipse
from skimage import io, color
from skimage import data
from skimage import transform
from skimage import img_as_float

In [ ]:
image_pydicom = pydicom.read_file('0.063-1_2.5x_View2.dcm')
image_arr = image_pydicom.pixel_array

image = image_arr

In [ ]:
def show_image(image_arr, title='Image'):
    image_voi = apply_voi_lut(image_arr, image_pydicom)
    
    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

In [ ]:
def save_image(image, filename='image.png'):
    plt.imshow(image, cmap='gray')
    plt.axis('off') 
    plt.savefig(filename, bbox_inches='tight', pad_inches=0)
    plt.close()

In [ ]:
show_image(image, 'Исходник')

## Analysis

In [ ]:
cum_image = None
images = []
images_after = []

def reset():
    global cum_image 
    cum_image = np.copy(image_arr)
    global images 
    images = [image, cum_image]
    global images_after
    images_after = []

reset()

In [ ]:
def accumulate():
    global images
    global images_after
    images[1] = images_after[1]

In [ ]:
def show_before_after(images, images_after, title_after=None, titles_before=['Original Image', 'Cumulative Image']):
    fig, axes = plt.subplots(len(images), 2, figsize=(12, 6 * len(images)))

    for i, (img, img_after) in enumerate(zip(images, images_after)):
        title_before = titles_before[i] if titles_before else f'Image {i+1}'
        title_after = title_after if title_after else f'Image {i+1} After'

        # Original Image
        axes[i, 0].imshow(img, cmap='gray')
        axes[i, 0].set_title(title_before)
        axes[i, 0].axis('off')

        # Processed Image
        axes[i, 1].imshow(img_after, cmap='gray')
        axes[i, 1].set_title(title_after)
        axes[i, 1].axis('off')

    plt.show()

## Contrast stretching

In [ ]:
print(image.shape)
print(np.min(image))
print(np.max(images))

In [ ]:
def contrast_stretching(image, min_out=0, max_out=255):
    min_val = np.min(image)
    max_val = np.max(image)
    
    if min_val == min_out and max_val == max_out:
        print("Image already satisfies requirements")
        return image
    
    return ((image - min_val) / (max_val - min_val) * (max_out - min_out) + min_out).astype(np.uint8)

images_after = list(map(contrast_stretching, images))
show_before_after(images, images_after, 'Contrast Stretched')

In [ ]:
# accumulate()

## Gamma correction

In [ ]:
def gamma_correction(image, gamma=1.0):
    image = image / 255.0 
    corrected = np.power(image, gamma) * 255
    return np.clip(corrected, 0, 255).astype(np.uint8)

images_after = list(map(lambda img: gamma_correction(img, gamma=.75), images))
show_before_after(images, images_after, 'Gamma Correction')

In [ ]:
# accumulate()

## Histogram Equalization

In [ ]:
from skimage import exposure

def equalize_histogram(image):
    return exposure.equalize_hist(image) * 255

images_after = list(map(equalize_histogram, images))
show_before_after(images, images_after, 'Equalize Histogram')

In [ ]:
save_image(images_after[0], 'result.jpg')

In [ ]:
# accumulate()

## Histogram projections

In [ ]:
def histogram_projection(image, proj_type='horizontal'):
    if proj_type == 'horizontal':
        return np.sum(image, axis=0)
    elif proj_type == 'vertical':
        return np.sum(image, axis=1)
    else:
        raise ValueError("Invalid projection type. Use 'horizontal' or 'vertical'.")

hist_proj_horizontal = [histogram_projection(img, 'horizontal') for img in images]
hist_proj_vertical = [histogram_projection(img, 'vertical') for img in images]

def show_histogram_projections(projections, titles=['Horizontal Projection', 'Vertical Projection']):
    plt.figure(figsize=(12, 6 * len(projections)))
    for i, (proj, title) in enumerate(zip(projections, titles)):
        plt.subplot(len(projections), 1, i+1)
        plt.plot(proj)
        plt.title(title)
        plt.xlabel('Pixel Position')
        plt.ylabel('Sum of Intensities')
    plt.show()

show_histogram_projections([hist_proj_horizontal[0], hist_proj_vertical[0]])

In [ ]:
image.shape

## CLAHE

In [ ]:
from skimage import exposure

def adaptive_histogram_equalization(image, clip_limit=0.03):
    clahe = exposure.equalize_adapthist(image, clip_limit=clip_limit)
    return np.clip(clahe * 255, 0, 255).astype(np.uint8)

images_after = list(map(adaptive_histogram_equalization, images))
show_before_after(images, images_after, 'CLAHE')

In [ ]:
# accumulate()

## Sharpening

In [ ]:
def sharpen_image(image):
    kernel = np.array([[0, -1, 0], [-1, 5,-1], [0, -1, 0]])
    return cv2.filter2D(image, -1, kernel)

images_after = list(map(sharpen_image, images))
show_before_after(images, images_after, 'Sharpened Image')

In [ ]:
# accumulate()